### Production Features Pipeline - CSV Version

This notebook is run daily from a Github Action. 

1. It scrapes the results from the previous day's games, performs feature engineering, and saves the results back to a csv file. This is an alternative version of the pipeline that DOES NOT utilize the Hopsworks.ai Feature Store and is less dependent on other platforms.

2. It scrapes the upcoming games for today, and saves the blank records back into the csv file so that they can be accessed by the model for the prediction.

**Note:**
There are two options for webscraping in this notebook. 
Set the 'WEBSCRAPER' variable to either 'SCRAPINGANT' or 'SELENIUM' to choose which version to run.

1. SCRAPINGANT: Uses a webscraping service with a Python API, ScrapingAnt, which handles all the proxy server issues, but does require an account. The free account allows for 1000 page requests, which is more than enough for this project. Proxies are required when running this notebook from a Github Action or otherwise key data will fail to be scraped from NBA.com. 

2. SELENIUM: This option does not currently integrate proxy servers into the webscraping process, which can cause issues when scraping from certain locations, in particular Github Actions. For occasional use from local machines, this option may work fine, but you may need to setup a proxy server.

In [1]:
# select web scraper; 'SCRAPINGANT' or 'SELENIUM'
# SCRAPINGANT requires a subscription but includes a proxy server

WEBSCRAPER = 'SCRAPINGANT'
#WEBSCRAPER = 'SELENIUM'

In [2]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

import json

import time

from pathlib import Path  #for Windows/Linux compatibility

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir('..') 

 
from src.webscraping import (
    get_new_games,
    activate_web_driver,
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)


DATAPATH = Path(r'data')

**Load API keys**

In [3]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

# if scrapingant is chosen then set the api key, otherwise load the selenium webdriver
if WEBSCRAPER == 'SCRAPINGANT':
    try:
        SCRAPINGANT_API_KEY = os.environ['SCRAPINGANT_API_KEY']
    except:
        raise Exception('Set environment variable SCRAPINGANT_API_KEY')
    driver = None
    
elif WEBSCRAPER == 'SELENIUM':
    driver = activate_web_driver('chromium')
    SCRAPINGANT_API_KEY = ""
    



**Scrape New Completed Games and Format Them**

In [4]:


df_new = get_new_games(SCRAPINGANT_API_KEY, driver)

if df_new.empty:
    print('No new games to process')

    # determine what season we are in currently
    today = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    if today.month >= 10:
        SEASON = today.year
    else:
        SEASON = today.year - 1
else:

    # get the SEASON of the last game in the database
    # this will used when constructing rows for prediction
    SEASON = df_new['SEASON'].max()

    df_new




Current month is 10
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Regular+Season&DateFrom=10/20/24&DateTo=10/27/24
0     1610612758
1     1610612746
2     1610612743
3     1610612738
4     1610612765
         ...    
67    1610612744
68    1610612750
69    1610612747
70    1610612738
71    1610612752
Length: 72, dtype: object
HOME_TEAM_ID 0     1610612743
1     1610612765
2     1610612766
3     1610612764
4     1610612741
5     1610612763
6     1610612750
7     1610612759
8     1610612756
9     1610612747
10    1610612757
11    1610612747
12    1610612762
13    1610612745
14    1610612749
15    1610612737
16    1610612752
17    1610612761
18    1610612739
19    1610612753
20    1610612758
21    1610612743
22    1610612742
23    1610612764
24    1610612765
25    1610612761
26    1610612748
27    1610612755
28    1610612737
29    1610612745
30    1610612740
31    1610612762
32    1610612746
33    1610612757
34    1610612747
35    1610612738
Name: HOME_TEAM_ID, dtype: obje

**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(SCRAPINGANT_API_KEY, driver)

if matchups is None:
    print('No games today')
else:
    print(matchups)
    print(game_ids)


Sat
Sun
[['1610612755', '1610612754'], ['1610612749', '1610612751'], ['1610612740', '1610612757'], ['1610612737', '1610612760'], ['1610612746', '1610612744']]
['22400097', '22400098', '22400099', '22400100', '22400101']


**Close Webdriver**

In [6]:
if WEBSCRAPER == 'SELENIUM':
    driver.close() 

**Check if anything is going on in the season**

In [7]:
if (df_new.empty) and (matchups is None):
    print('No new games to process')
    exit()
    

**Create Rows for Today's Games with Empty Stats**

In [8]:
# reformat today's matchups to the new games dataframe

if matchups is None:
    print('No games going on. Nothing to do.')
    exit()    

else:

    df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
    for i, matchup in enumerate(matchups):
        game_details = {'HOME_TEAM_ID': matchup[1], 
                        'VISITOR_TEAM_ID': matchup[0], 
                        'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                        'GAME_ID': int(game_ids[i]),                       
                        'SEASON': SEASON,
                        } 
        game_details_df = pd.DataFrame(game_details, index=[i])
        # append to new games dataframe
        df_today = pd.concat([df_today, game_details_df], ignore_index = True)

    #blank rows will be filled with 0 to prevent issues with feature engineering
    df_today = df_today.fillna(0) 

    df_today



**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [9]:


df_old = pd.read_csv(DATAPATH / 'games.csv')

df_old


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,104.0,0.398,0.760,...,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,101.0,0.443,0.933,...,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,108.0,0.412,0.813,...,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,122.0,0.484,0.933,...,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,115.0,0.551,0.750,...,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28813,2024-10-23 00:00:00,22400066.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,109.0,41.800,70.400,...,18.0,49.0,NaN,124.0,53.800,71.000,43.200,27.0,46.0,0
28814,2024-10-23 00:00:00,22400065.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,97.0,39.000,75.900,...,24.0,41.0,NaN,116.0,42.700,72.700,36.700,28.0,57.0,0
28815,2024-10-23 00:00:00,22400067.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,106.0,41.900,75.900,...,25.0,38.0,NaN,136.0,58.600,76.900,46.700,29.0,36.0,0
28816,2024-10-23 00:00:00,22400063.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,109.0,43.200,85.700,...,22.0,38.0,NaN,115.0,50.600,88.000,29.000,32.0,39.0,0


**Update Yesterday's Matchup Predictions with New Final Results**

In [10]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results


# one approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)


# save the new games to the database
df_old.to_csv(DATAPATH / 'games.csv', index=False)

df_old

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,104.0,0.398,0.760,...,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,101.0,0.443,0.933,...,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,108.0,0.412,0.813,...,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,122.0,0.484,0.933,...,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,115.0,0.551,0.750,...,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28849,2024-10-23 00:00:00,22400070.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,124.0,47.100,75.600,...,21.0,49.0,NaN,126.0,47.300,71.900,37.800,28.0,41.0,0
28850,2024-10-23 00:00:00,22400071.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,113.0,45.700,70.000,...,27.0,51.0,NaN,116.0,48.700,73.500,39.500,25.0,42.0,0
28851,2024-10-23 00:00:00,22400072.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,104.0,39.100,80.000,...,21.0,42.0,NaN,140.0,51.600,80.000,41.700,38.0,57.0,0
28852,2024-10-22 00:00:00,22400062.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,110.0,44.200,84.000,...,22.0,46.0,NaN,103.0,41.200,74.100,31.700,17.0,47.0,1


**Add Today's Matchups for Feature Engineering**

In [11]:
if matchups is None:
    print('No games today')
    df_combined = df_old
else:
    df_combined = pd.concat([df_old, df_today], ignore_index = True)
    df_combined

**Data Processing**

In [12]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2022-03-12,22101005.0,1610612748.0,1610612750.0,2021.0,104.0,0.398,0.760,0.333,23.0,53.0,113.0,0.422,0.875,0.357,21.0,46.0,0.0,0,0.0
1,2022-03-12,22101006.0,1610612741.0,1610612739.0,2021.0,101.0,0.443,0.933,0.429,20.0,46.0,91.0,0.419,0.824,0.208,19.0,40.0,1.0,0,1.0
2,2022-03-12,22101007.0,1610612759.0,1610612754.0,2021.0,108.0,0.412,0.813,0.324,28.0,52.0,119.0,0.489,1.000,0.389,23.0,47.0,0.0,0,0.0
3,2022-03-12,22101008.0,1610612744.0,1610612749.0,2021.0,122.0,0.484,0.933,0.400,33.0,55.0,109.0,0.413,0.696,0.386,27.0,39.0,1.0,0,1.0
4,2022-03-12,22101009.0,1610612743.0,1610612761.0,2021.0,115.0,0.551,0.750,0.407,32.0,39.0,127.0,0.471,0.760,0.387,28.0,50.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28854,2024-10-27,22400097.0,1610612754,1610612755,2024.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0,0.0
28855,2024-10-27,22400098.0,1610612751,1610612749,2024.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0,0.0
28856,2024-10-27,22400099.0,1610612757,1610612740,2024.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0,0.0
28857,2024-10-27,22400100.0,1610612760,1610612737,2024.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0,0.0


**Feature Engineering**

In [13]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)



#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')

# save file
df_combined.to_csv(DATAPATH / 'games_engineered.csv', index=False)


df_combined


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28 00:00:00+00:00,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28 00:00:00+00:00,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28 00:00:00+00:00,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29 00:00:00+00:00,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29 00:00:00+00:00,20300008,1610612765,1610612754,2003,87,0.392090,0.742188,0.333008,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27039,2024-10-27 00:00:00+00:00,22400098,1610612751,1610612749,2024,0,0.000000,0.000000,0.000000,0,...,-0.710938,-1.439583,-2.666667,-2.428571,-2.0,-1.666667,-3.666667,2.142857,2.3,2.133333
27040,2024-10-27 00:00:00+00:00,22400097,1610612754,1610612755,2024,0,0.000000,0.000000,0.000000,0,...,-0.043750,-0.448958,9.000000,7.428571,7.2,5.333333,-1.333333,-2.142857,-2.6,-4.066667
27041,2024-10-27 00:00:00+00:00,22400100,1610612760,1610612737,2024,0,0.000000,0.000000,0.000000,0,...,-1.020312,-1.581250,-3.666667,-5.142857,-4.9,-2.200000,5.333333,1.000000,0.8,1.666667
27042,2024-10-27 00:00:00+00:00,22400101,1610612744,1610612746,2024,0,0.000000,0.000000,0.000000,0,...,6.840625,5.101042,4.000000,8.285714,7.9,6.866667,6.333333,5.714286,3.4,1.200000
